In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- Veri Yükleme ve Ön İşleme Adımlarını Tekrarlama ---

# Veriyi yükle
file_path = '../data/raw/cic_ids_2017.csv'
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()

# 'Label' sütununu sayısal yapalım (Bu modelleme için gerekli)
# DDoS -> 1 (Anomali), BENIGN -> 0 (Normal)
df['Label'] = df['Label'].apply(lambda x: 1 if x == 'DDoS' else 0)

# Özellikleri (X) ve hedefi (y) ayır
X = df.drop('Label', axis=1)
y = df['Label']

# Veriyi ayır
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Sayısal sütunları ölçekle
numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

print("Veri ön işleme tamamlandı. Modellemeye hazır.")
print("Eğitim seti boyutu:", X_train.shape)
print("Test seti boyutu:", X_test.shape)

ValueError: Input X contains infinity or a value too large for dtype('float64').